<a href="https://colab.research.google.com/github/ribeirod/publico/blob/main/10_PySpark_com_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PySpark com SQL**

Abaixo segue exemplos de consultas SQL usando o PySpark. O objetivo é mostrar a filtragem dos dados por meio do SQL e não do PySpark DataFrame (similar ao Pandas DataFrame).

Os dados podem ser baixados aqui: https://www.kaggle.com/datasets/adaoduque/campeonato-brasileiro-de-futebol


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=8fd4121eb8e9eff21c690bdb02afec6739ddeea8c80dab46dcd76a2c68d8216b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
# importando bibliotecas
from google.colab import drive
drive.mount('/content/drive')
from pyspark.sql import SparkSession

Mounted at /content/drive


In [ ]:
# criacao da Spark Session
spark = SparkSession.builder.appName("exemplo_sql").getOrCreate()

# leitura de arquivo CSV para criacao de DataFrame
dfCartoes = spark.read.csv("/content/drive/MyDrive/ProjectsDS/privado/10-PySparkSQL/data/campeonato-brasileiro-cartoes.csv",
                    header=True, inferSchema=True)
dfJogos = spark.read.csv("/content/drive/MyDrive/ProjectsDS/privado/10-PySparkSQL/data/campeonato-brasileiro-full.csv",
                    header=True, inferSchema=True)
dfEstatisticas = spark.read.csv("/content/drive/MyDrive/ProjectsDS/privado/10-PySparkSQL/data/campeonato-brasileiro-estatisticas-full.csv",
                    header=True, inferSchema=True)
dfGols = spark.read.csv("/content/drive/MyDrive/ProjectsDS/privado/10-PySparkSQL/data/campeonato-brasileiro-gols.csv",
                    header=True, inferSchema=True)


In [ ]:
# registrando os DataFrames como tabelas temporária
dfCartoes.createOrReplaceTempView("tabela_cartoes")
dfJogos.createOrReplaceTempView("tabela_jogos")
dfEstatisticas.createOrReplaceTempView("tabela_estatisticas")
dfGols.createOrReplaceTempView("tabela_gols")

In [ ]:
# mostrando as 5 primeiras linhas de cada tabela
resultado = spark.sql("SELECT * FROM tabela_cartoes LIMIT 5")
print("tabela_cartoes")
resultado.show()
resultado = spark.sql("SELECT * FROM tabela_jogos LIMIT 5")
print("tabela_jogos")
resultado.show()
resultado = spark.sql("SELECT * FROM tabela_estatisticas LIMIT 5")
print("tabela_estatisticas")
resultado.show()
resultado = spark.sql("SELECT * FROM tabela_gols LIMIT 5")
print("tabela_gols")
resultado.show()

tabela_cartoes
+----------+------+-------------+-------+--------------------+----------+----------+------+
|partida_id|rodata|        clube| cartao|              atleta|num_camisa|   posicao|minuto|
+----------+------+-------------+-------+--------------------+----------+----------+------+
|      4607|     1|  Figueirense|Amarelo|Paulo Roberto da ...|        28|      NULL|    66|
|      4607|     1|  Figueirense|Amarelo|       Thiago Heleno|         4|  Zagueiro|    44|
|      4608|     1|Internacional|Amarelo| Andrés D'Alessandro|        10|Meio-campo|    72|
|      4608|     1|      Vitoria|Amarelo|Marcelo Machado d...|        29|      NULL|    86|
|      4608|     1|      Vitoria|Amarelo|              Mansur|      NULL|  Zagueiro|    10|
+----------+------+-------------+-------+--------------------+----------+----------+------+

tabela_jogos
+---+------+----------+-------------------+-------------+-----------+-----------------+------------------+----------------+-----------------+--

**1) Selecione as partidas que ocorreram no Maracanã.**

In [ ]:
spark.sql("""SELECT j.data, j.mandante, j.visitante
          FROM tabela_jogos j
          WHERE j.arena LIKE '%Maracanã%'
          ORDER BY j.data""").show()

+----------+-----------+-------------+
|      data|   mandante|    visitante|
+----------+-----------+-------------+
|01/06/2006|Botafogo-RJ|  Figueirense|
|01/06/2008| Fluminense|     Flamengo|
|01/07/2023|   Flamengo|    Fortaleza|
|01/08/2010|   Flamengo|        Vasco|
|01/08/2013|Botafogo-RJ|      Vitoria|
|01/08/2015| Fluminense|       Gremio|
|01/09/2004| Fluminense|Internacional|
|01/09/2007|   Flamengo|        Sport|
|01/09/2010| Fluminense|    Palmeiras|
|01/09/2013|Botafogo-RJ|    Sao Paulo|
|01/09/2018|      Vasco|       Santos|
|01/09/2019|   Flamengo|    Palmeiras|
|01/10/2008| Fluminense|        Goias|
|01/10/2022|   Flamengo|   Bragantino|
|01/11/2003| Fluminense|  Ponte Preta|
|01/11/2006|   Flamengo|   Santa Cruz|
|01/11/2008|   Flamengo|   Portuguesa|
|01/11/2020|   Flamengo|    Sao Paulo|
|01/12/2013|      Vasco|      Nautico|
|01/12/2018|   Flamengo| Athletico-PR|
+----------+-----------+-------------+
only showing top 20 rows



**2) Selecione nome e saldo de gols de todos os times mandantes do Estado de SP.**

In [ ]:
spark.sql("""SELECT j.mandante, SUM(j.mandante_Placar) AS total_gols
          FROM tabela_jogos j
          WHERE j.mandante_Estado LIKE '%SP%'
          GROUP BY j.mandante
          ORDER BY total_gols DESC;""").show()

+---------------+----------+
|       mandante|total_gols|
+---------------+----------+
|         Santos|       728|
|      Sao Paulo|       685|
|      Palmeiras|       641|
|    Corinthians|       586|
|    Ponte Preta|       256|
|    Sao Caetano|       120|
|     Bragantino|       119|
|     Portuguesa|        90|
|        Guarani|        83|
|        Barueri|        35|
|    Santo Andre|        29|
|Gremio Prudente|        25|
+---------------+----------+



**3) Selecione ano e saldo de gols de todos os times mandantes do Estado de SP.**

In [ ]:
spark.sql("""SELECT RIGHT(j.data, 4) AS ano, SUM(j.mandante_Placar) AS total_gols
          FROM tabela_jogos j
          WHERE j.mandante_Estado LIKE '%SP%'
          GROUP BY ano
          ORDER BY total_gols DESC;""").show()

+----+----------+
| ano|total_gols|
+----+----------+
|2004|       271|
|2003|       252|
|2005|       229|
|2009|       200|
|2006|       186|
|2021|       179|
|2015|       176|
|2010|       172|
|2012|       171|
|2016|       160|
|2022|       152|
|2017|       147|
|2023|       144|
|2008|       142|
|2019|       132|
|2013|       119|
|2011|       119|
|2007|       115|
|2018|       114|
|2014|       110|
+----+----------+
only showing top 20 rows



**4) Selecione data e local de todas as partidas jogadas pelo Palmeiras**

In [ ]:
spark.sql("""SELECT j.data, j.arena
          FROM tabela_jogos j
          WHERE j.mandante LIKE '%Palmeiras%' OR j.visitante LIKE '%Palmeiras%'
          ORDER BY j.data DESC;""").show()

+----------+--------------------+
|      data|               arena|
+----------+--------------------+
|31/10/2021|     Arena do Grêmio|
|31/08/2011|            Engenhão|
|31/08/2008|       Kyocera Arena|
|31/07/2021|             Morumbi|
|31/07/2016|     Luso-Brasileiro|
|31/07/2005|       Pacaembu*(PF)|
|31/05/2015|   Neo Química Arena|
|31/05/2009|       Arena Barueri|
|31/05/2009|     Palestra Itália|
|31/05/2006|            Maracanã|
|30/11/2021|      Arena Pantanal|
|30/11/2008|            Barradão|
|30/10/2019|      Allianz Parque|
|30/10/2017|      Allianz Parque|
|30/10/2011|     Arena do Jacaré|
|30/10/2010|       Arena Barueri|
|30/10/2005|            Mineirão|
|30/10/2004|       Bento Freitas|
|30/09/2018|Estádio Municipal...|
|30/09/2017|      Allianz Parque|
+----------+--------------------+
only showing top 20 rows



**5) Para cada atleta que já fez gol, selecione o nome, time, nº camisa e estado do time para o qual joga ou já jogou.**

In [ ]:
spark.sql("""SELECT c.atleta, c.clube, c.num_camisa, j.mandante_Estado As clube_estado
          FROM tabela_cartoes c
          LEFT JOIN tabela_jogos j
          ON c.clube = j.mandante
          WHERE c.atleta IS NOT NULL
          GROUP BY c.atleta, c.clube, c.num_camisa, clube_estado
          ORDER BY c.atleta;""").show()

+--------------------+-------------+----------+------------+
|              atleta|        clube|num_camisa|clube_estado|
+--------------------+-------------+----------+------------+
|      Abel Hernández|   Fluminense|        32|          RJ|
|      Abel Hernández|Internacional|        99|          RS|
|      Abner Vinícius| Athletico-PR|        16|          PR|
|               Abuda|  Ponte Preta|        18|          SP|
|               Abuda|  Chapecoense|        22|          SC|
|               Abuda|  Ponte Preta|         5|          SP|
|Adalberto dos San...|   America-MG|         4|          MG|
|Adalberto dos San...|    Fortaleza|        29|          CE|
|           Ademilson|    Sao Paulo|        19|          SP|
|              Ademir|   America-MG|        10|          MG|
|              Ademir|   America-MG|        15|          MG|
|              Ademir|   America-MG|        19|          MG|
|              Ademir|  Atletico-MG|        19|          MG|
|              Ademir|  

**6) Selecione os times que não jogaram nenhuma partida nos meses de julho e agosto de 2021.**

In [ ]:
spark.sql("""SELECT j.data, j.mandante, j.visitante
          FROM tabela_jogos j
          WHERE EXTRACT(YEAR FROM to_date(j.data,'dd/MM/yyyy')) <> 2021
          AND EXTRACT(MONTH FROM to_date(j.data,'dd/MM/yyyy')) <> 7
          AND EXTRACT(MONTH FROM to_date(j.data,'dd/MM/yyyy')) <> 8""").show()

+----------+-------------+-------------+
|      data|     mandante|    visitante|
+----------+-------------+-------------+
|29/03/2003|      Guarani|        Vasco|
|29/03/2003| Athletico-PR|       Gremio|
|30/03/2003|     Flamengo|     Coritiba|
|30/03/2003|        Goias|     Paysandu|
|30/03/2003|Internacional|  Ponte Preta|
|30/03/2003|     Criciuma|   Fluminense|
|30/03/2003|    Juventude|    Sao Paulo|
|30/03/2003|    Fortaleza|        Bahia|
|30/03/2003|     Cruzeiro|  Sao Caetano|
|30/03/2003|      Vitoria|  Figueirense|
|30/03/2003|       Santos|       Parana|
|30/03/2003|  Corinthians|  Atletico-MG|
|05/04/2003|   Fluminense|    Fortaleza|
|05/04/2003|  Atletico-MG|       Santos|
|05/04/2003|     Coritiba|Internacional|
|05/04/2003|       Gremio|      Guarani|
|06/04/2003|        Bahia|     Flamengo|
|06/04/2003|  Figueirense|  Corinthians|
|06/04/2003|     Paysandu|      Vitoria|
|06/04/2003|  Ponte Preta|    Juventude|
+----------+-------------+-------------+
only showing top

**7) Selecionar a quantidade de jogos ocorridos no mês de julho de 2021 que tiveram cartão amarelo.**

In [ ]:
spark.sql("""SELECT COUNT(*) AS QtdJogosCartaoAmarelo2021
          FROM tabela_jogos j JOIN tabela_cartoes c
          ON j.ID = c.partida_id
          WHERE c.cartao = 'Amarelo' AND EXTRACT(MONTH FROM to_date(j.data,'dd/MM/yyyy')) = 7
          AND EXTRACT(YEAR FROM to_date(j.data,'dd/MM/yyyy')) = 2021;""").show()

+-------------------------+
|QtdJogosCartaoAmarelo2021|
+-------------------------+
|                      267|
+-------------------------+



**8) Selecione os nomes do atleta, do clube, do cartão e a posição do atleta de todas as partidas que tiveram gol do tipo Penalty e a posição do atleta é diferente de NULL.**

In [ ]:
spark.sql("""SELECT atleta, clube, cartao, posicao
              FROM tabela_cartoes
              WHERE atleta NOT IN (SELECT atleta FROM tabela_gols
                                   WHERE tipo_de_gol = 'Penalty')
              AND posicao IS NOT NULL; """).show()

+--------------------+-------------+-------+----------+
|              atleta|        clube| cartao|   posicao|
+--------------------+-------------+-------+----------+
|              Mansur|      Vitoria|Amarelo|  Zagueiro|
|Juan Silveira dos...|Internacional|Amarelo|  Zagueiro|
|Carlos Emiliano P...|     Coritiba|Amarelo|  Zagueiro|
|Cristian Chagas T...|        Bahia|Amarelo|  Zagueiro|
|             Rhayner|        Bahia|Amarelo|  Atacante|
|Willian Gomes de ...|     Cruzeiro|Amarelo|  Atacante|
|                Pará|        Bahia|Amarelo|  Zagueiro|
|               Fahel|        Bahia|Amarelo|Meio-campo|
|   Paulo César Tinga|     Cruzeiro|Amarelo|Meio-campo|
|Nílton Ferreira J...|     Cruzeiro|Amarelo|Meio-campo|
|      Marcelo Mattos|  Botafogo-RJ|Amarelo|Meio-campo|
|      Álvaro Pereira|    Sao Paulo|Amarelo|  Zagueiro|
|Juan Carlos Ferreyra|  Botafogo-RJ|Amarelo|  Atacante|
|       Paulinho Dias| Athletico-PR|Amarelo|Meio-campo|
|Ramiro Moschen Be...|       Gremio|Amarelo|Meio